<a href="https://colab.research.google.com/github/DavidB363/file-store/blob/master/Lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# My Recommender System

import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [41]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv')
df=df.drop(df.columns[0],1)
print(df)

       -7.82   8.79  -9.66  -8.16  -7.52  ...  99.22  -5.63  99.23  99.24  99.25
0       4.08  -0.29   6.36   4.37  -2.38  ...  -2.14   3.06   0.34  -4.32   1.07
1      99.00  99.00  99.00  99.00   9.03  ...  99.00  99.00  99.00  99.00  99.00
2      99.00   8.35  99.00  99.00   1.80  ...  99.00  99.00  99.00  99.00  99.00
3       8.50   4.61  -4.17  -5.39   1.36  ...   1.55   3.11   6.55   1.80   1.60
4      -6.17  -3.54   0.44  -8.50  -7.09  ...  -3.35   0.05  -9.08  -5.05  -3.45
...      ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24977   0.44   7.43   9.08   2.33   3.20  ...   9.03   6.55   8.69   8.79   7.43
24978   9.13  -8.16   8.59   9.08   0.87  ...  -8.20  -7.23  -8.59   9.13   8.45
24979  99.00  99.00  99.00  99.00  -7.77  ...  99.00  99.00  99.00  99.00  99.00
24980  99.00  99.00  99.00  99.00  -9.71  ...  99.00  99.00  99.00  99.00  99.00
24981   2.43   2.67  -3.98   4.27  -2.28  ...  99.00  99.00  99.00  99.00  99.00

[24982 rows x 100 columns]


In [42]:
arr = df.values
print(arr.shape)
#print(arr_full)

(24982, 100)


In [43]:
rated = np.where(arr!=99)
print(rated[0].shape)
print(rated[0])
print(rated[1])
print(len(rated[0]), rated[1].shape)

(1810381,)
[    0     0     0 ... 24981 24981 24981]
[ 0  1  2 ... 69 70 86]
1810381 (1810381,)


In [0]:

def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  print('idx',idx)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [45]:
new_arr, idx = replace(arr, 0.1)
print(idx)

idx [1769524 1509023 1777710 ... 1389921 1535663  454779]
(array([24422, 20816, 24538, ..., 19184, 21181,  6308]), array([ 4, 14, 23, ..., 49,  5,  0]))


In [0]:
# arr[idx[0][0], idx[1][0]]

In [0]:
# new_arr[idx[0][0], idx[1][0]]

In [0]:
# Set the number of latent factors to 2.

n_latent_factors = 2

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
# latent_item_features

In [0]:
# latent_user_preferences

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
#def train(user_id, item_id, rating, alpha=0.01):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    



#def sgd(iterations=300000):
def sgd(iterations=10):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = [] 
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        # if (iteration % 10000) == 0:
        if (iteration % 1) == 0:
            print(mse)

In [52]:
sgd(10)  # Note how the MSE decreases with the number of iterations


26.200693919286657
25.271343905914478
24.983574695050837
24.764657831085323
24.552591386759648
24.33895371949194
24.122981218739138
23.905515368972633
23.687854743603605
23.471439333509874


In [0]:
predictions = np.zeros(shape=(latent_user_preferences.shape[0], latent_item_features.shape[0]))
#print(user_features_weights)
#print(item_features_weights)
for user_id in range(latent_user_preferences.shape[0]):
    for item_id in range(latent_item_features.shape[0]):
        predictions[user_id, item_id] =  predict_rating(user_id, item_id)

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [57]:
comparison_data

,-7.82,8.79,-9.66,-8.16,-7.52,-8.5,-9.85,4.17,-8.98,-4.76,-8.5.1,-6.75,-7.18,8.45,-7.18.1,-7.52.1,-7.43,-9.81,-9.85.1,-9.85.2,-9.37,1.5,-4.37,-9.81.1,-8.5.2,1.12,7.82,2.86,9.13,-7.43.1,2.14,-4.08,-9.08,7.82.1,5.05,4.95,-9.17,-8.4,-8.4.1,-8.4.2,...,8.59,3.59,-6.84,-9.03.1,2.82.1,-1.36,-9.08.2,8.3.1,5.68,-4.81,99,99.1,99.2,99.3,99.4,99.5,99.6,-9.42.1,99.7,99.8,99.9,-7.72,99.10,99.11,99.12,99.13,99.14,99.15,99.16,99.17,2.82.2,99.18,99.19,99.20,99.21,99.22,-5.63,99.23,99.24,99.25
0,"(4.08, 0.9448574539576066)","(-0.29, 0.6440890477409144)","(6.36, 0.6274782602486988)","(4.37, -0.7066126453595443)","(-2.38, 0.7289823557184643)","(99.0, 1.707896579107285)","(-0.73, -0.05914275758600113)","(-5.34, -0.33382726361910964)","(8.88, -0.09492383331911333)","(99.0, 1.229864556878037)","(6.75, 1.7823878652861112)","(8.64, 1.7128652941868039)","(4.42, -1.1433044696110137)","(7.43, 1.6897925025545262)","(4.56, -1.1707629446450512)","(-0.97, -2.1558577300281545)","(4.66, -0.5298464935304648)","(-0.68, -0.3762586540303654)","(3.3, 0.45947973151700494)","(-1.21, -0.3371732426452665)","(99.0, 2.2050666105681698)","(8.64, 1.116185404471469)","(8.35, 0.489893827055087)","(9.17, -0.8621491059843255)","(0.05, 0.8126437493759807)","(7.57, 1.5933838605524921)","(4.71, 2.851152299066615)","(0.87, 1.5968602725807735)","(-0.39, 2.7113863759559433)","(6.99, 0.1519692528957261)","(6.5, 2.239223957581145)","(-0.92, 2.7904535120151093)","(7.14, -0.6396259380236937)","(9.03, 1.216118964933425)","(-1.8, 2.743387288602386)","(0.73, 2.900147821367556)","(7.09, -0.7439806340667887)","(3.4, 1.3755176483041707)","(-0.87, 1.39400205118112)","(7.91, 1.279902773085959)",...,"(-6.7, 2.3721919235850564)","(-3.35, 2.6798674653339956)","(-9.03, 0.6622585672061823)","(4.47, -0.19715470664500623)","(4.08, 2.3463183356045256)","(-3.83, 2.390438453767801)","(99.0, -0.21025380728169316)","(1.12, 2.4917016902913196)","(0.78, 2.5088491301381124)","(7.52, 0.8740752881730227)","(-5.0, -0.13953549186484313)","(2.77, 2.51763358489192)","(8.3, 1.376975938626868)","(7.77, -0.5988425766556826)","(7.33, 0.23325175146164545)","(6.21, 2.2924988529585555)","(7.72, 1.103232595402698)","(8.98, 1.904868310529403)","(8.64, 0.690245773125012)","(8.2, 1.5211053515180348)","(99.0, 1.8614895454084606)","(4.85, 1.0823896892592284)","(4.85, 2.106165510959426)","(6.07, 1.0979846174252585)","(8.98, 1.1552938156485575)","(4.51, 0.6206943858013079)","(-0.05, 1.850368978676461)","(3.69, 1.9691960885846223)","(4.56, 2.9637766709070035)","(0.58, 0.952963215543842)","(99.0, 1.8356342041429459)","(-4.95, 1.437092274093741)","(-0.29, 2.180824007186885)","(7.86, 1.2494919305091075)","(99.0, 1.2096869916347888)","(-2.14, 1.471715636318872)","(3.06, 1.5572646023607064)","(0.34, 1.0752598025457745)","(-4.32, 0.5748393505414437)","(1.07, 1.477115637422665)"
1,"(99.0, 1.666997727711148)","(99.0, 1.154120166701493)","(99.0, 1.1174426351730722)","(99.0, -1.2790270690922119)","(9.03, 1.324176082877958)","(9.27, 3.0405602943689183)","(9.03, -0.10678867962877511)","(9.27, -0.5999714174701449)","(99.0, -0.17305146549925657)","(99.0, 2.1787975049409165)","(7.33, 3.159745610647709)","(7.57, 3.057552082394003)","(9.37, -2.031950071555076)","(6.17, 3.0224859269905657)","(-6.36, -2.075034039659845)","(-6.89, -3.8231881841691617)","(-7.86, -0.9360888795493257)","(9.03, -0.6962396355342252)","(9.03, 0.81556155509424)","(9.03, -0.5973732813437498)","(99.0, 3.9397553875978533)","(99.0, 1.9815816595212041)","(8.25, 0.874550024427097)","(99.0, -1.5183994443933035)","(99.0, 1.445902835046733)","(7.48, 2.8462377723945)","(7.28, 5.077534275994574)","(7.28, 2.8278657630944215)","(8.93, 4.832373838058077)","(99.0, 0.2665207951538857)","(6.17, 3.9949837991888053)","(7.28, 4.955828477422237)","(99.0, -1.1593479734731706)","(99.0, 2.157240247729886)","(8.98, 4.8836484072283834)","(7.33, 5.170613260631917)","(99.0, -1.3384351877794674)","(6.17, 2.444902832110891)","(9.08, 2.479104348514466)","(7.33, 2.274092283612898)",...,"(6.46, 4.221969453